In [1]:
%run utils.ipynb

In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression as LR
from sklearn.metrics import mean_squared_error

In [3]:
pd.set_option("display.max_rows",None)
pd.set_option("display.max_columns",None)

In [4]:
df=pd.read_csv("../data/train.csv")

In [5]:
#drop some useless columns
cols_to_drop=["Id","PID"]
df=df.drop(columns=cols_to_drop)

In [6]:
df.isnull().mean().sort_values(ascending=False)*100

Pool QC            99.561190
Misc Feature       96.830814
Alley              93.174061
Fence              80.497318
Fireplace Qu       48.756704
Lot Frontage       16.089712
Garage Finish       5.558264
Garage Qual         5.558264
Garage Yr Blt       5.558264
Garage Cond         5.558264
Garage Type         5.509508
Bsmt Exposure       2.827889
BsmtFin Type 2      2.730375
Bsmt Cond           2.681619
Bsmt Qual           2.681619
BsmtFin Type 1      2.681619
Mas Vnr Area        1.072647
Mas Vnr Type        1.072647
Bsmt Full Bath      0.097513
Bsmt Half Bath      0.097513
Garage Area         0.048757
Garage Cars         0.048757
Total Bsmt SF       0.048757
Bsmt Unf SF         0.048757
BsmtFin SF 2        0.048757
BsmtFin SF 1        0.048757
3Ssn Porch          0.000000
Screen Porch        0.000000
Full Bath           0.000000
Half Bath           0.000000
Bedroom AbvGr       0.000000
Kitchen AbvGr       0.000000
Kitchen Qual        0.000000
TotRms AbvGrd       0.000000
Functional    

#### Imputesome missing values

In [7]:
cols_to_impute_with_none=["Pool QC","Misc Feature","Alley","Fence","Fireplace Qu",
                          "Garage Finish","Garage Qual","Garage Cond","Garage Type",
                         "Bsmt Exposure","BsmtFin Type 2","Bsmt Cond","Bsmt Qual","BsmtFin Type 1",
                         "Mas Vnr Type"]

cols_to_impute_with_zero=["Garage Yr Blt","Mas Vnr Area","Bsmt Full Bath","Bsmt Half Bath","Garage Area",
                         "Garage Cars","Total Bsmt SF","Bsmt Unf SF","BsmtFin SF 2","BsmtFin SF 1"]

In [8]:
for i in cols_to_impute_with_none:
    df[i]=df[i].fillna("None")
    
for j in cols_to_impute_with_zero:
    df[j]=df[j].fillna(0)

In [9]:
#Replace Lot Frontage with the median of the Neighborhood
df['Lot Frontage']=df.groupby(by="Neighborhood")['Lot Frontage'].transform(lambda x: x.fillna(x.median()))

In [10]:
#Should show as 0, to ensure no more nulls
df.isnull().sum().sum()

3

In [13]:
df[df['Lot Frontage'].isnull()]

,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,SalePrice
463,120,RM,NaN,9763,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,GrnHill,Norm,Norm,TwnhsE,1Story,7,5,1998,1998,Gable,CompShg,Wd Sdng,Wd Sdng,BrkFace,239.0,Gd,TA,PConc,Gd,TA,No,GLQ,1102.0,ALQ,72.0,328.0,1502.0,GasA,Ex,Y,SBrkr,1502,0,0,1502,1.0,0.0,1,1,1,1,Gd,5,Typ,1,TA,Attchd,1998.0,Fin,1.0,286.0,TA,TA,Y,0,0,0,0,0,0,None,None,None,0,9,2007,WD,330000
1121,160,RH,NaN,3612,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,Landmrk,Norm,Norm,Twnhs,2Story,6,5,1993,1994,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,GLQ,338.0,Unf,0.0,292.0,630.0,GasA,Ex,Y,SBrkr,630,690,0,1320,0.0,0.0,2,1,3,1,TA,5,Typ,0,None,Detchd,1993.0,Unf,2.0,484.0,TA,TA,Y,0,48,0,0,0,0,None,MnPrv,None,0,6,2006,WD,137000
1211,120,RM,NaN,8239,Pave,None,IR1,Lvl,AllPub,Inside,Gtl,GrnHill,Norm,Norm,TwnhsE,1Story,7,5,1986,1986,Gable,CompShg,BrkFace,Wd Sdng,None,0.0,Gd,TA,CBlock,None,None,None,None,0.0,None,0.0,0.0,0.0,GasA,Gd,Y,SBrkr,1295,0,0,1295,0.0,0.0,2,0,2,1,Gd,5,Typ,0,None,Attchd,1986.0,RFn,1.0,312.0,TA,TA,Y,0,0,0,0,0,0,None,None,None,0,11,2006,WD,230000


In [14]:
df['Neighborhood'].value_counts()

NAmes      310
CollgCr    180
OldTown    163
Edwards    143
Somerst    130
NridgHt    122
Gilbert    116
Sawyer     111
NWAmes      87
SawyerW     87
Mitchel     82
BrkSide     76
Crawfor     71
IDOTRR      69
NoRidge     48
Timber      48
StoneBr     38
SWISU       32
ClearCr     27
MeadowV     24
Blmngtn     22
BrDale      19
Veenker     17
NPkVill     17
Blueste      6
Greens       3
GrnHill      2
Landmrk      1
Name: Neighborhood, dtype: int64

In [11]:
df.isnull().mean().sort_values(ascending=False)*100

Lot Frontage       0.14627
MS SubClass        0.00000
Bedroom AbvGr      0.00000
Garage Type        0.00000
Fireplace Qu       0.00000
Fireplaces         0.00000
Functional         0.00000
TotRms AbvGrd      0.00000
Kitchen Qual       0.00000
Kitchen AbvGr      0.00000
Half Bath          0.00000
Garage Finish      0.00000
Full Bath          0.00000
Bsmt Half Bath     0.00000
Bsmt Full Bath     0.00000
Gr Liv Area        0.00000
Low Qual Fin SF    0.00000
2nd Flr SF         0.00000
1st Flr SF         0.00000
Garage Yr Blt      0.00000
Garage Cars        0.00000
Central Air        0.00000
Pool Area          0.00000
Sale Type          0.00000
Yr Sold            0.00000
Mo Sold            0.00000
Misc Val           0.00000
Misc Feature       0.00000
Fence              0.00000
Pool QC            0.00000
Screen Porch       0.00000
Garage Area        0.00000
3Ssn Porch         0.00000
Enclosed Porch     0.00000
Open Porch SF      0.00000
Wood Deck SF       0.00000
Paved Drive        0.00000
G